# Jupyter NoteBook del Tutorial

## Importar librerias

In [1]:
import pyodbc
import pandas as pd
from sqlalchemy import text

## Establecer conexión con la instancia de SQL Server

In [18]:
server = 'SANCHEZJAIME\\VIRTAMINTIC'
database = 'films_db'
username = 'equipo10'
password = '12345678'
driver = 'ODBC Driver 17 for SQL Server'

# Crear la cadena de conexión

conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password}"
)

try:
    # Establecer la conexión
    conn = pyodbc.connect(conn_str)
    conn.autocommit = True
    cursor = conn.cursor()
    
    # Ejecutar una consulta
    print("Conexión exitosa")
    
except Exception as e:
    print("Error en la conexión:", e)

Conexión exitosa


## Crear Tabla movies

In [8]:
create_table_query = """
                    CREATE TABLE movies (
                    id INT PRIMARY KEY,
                    title VARCHAR(255) NOT NULL,
                    release_year INT NULL,
                    country VARCHAR(100) NULL,
                    duration INT NULL,
                    language VARCHAR(100) NULL,
                    certification VARCHAR(50) NULL,
                    gross DECIMAL(18,2) NULL,
                    budget DECIMAL(18,2) NULL
                    );
"""
try:
    cursor.execute(create_table_query)
    print("Query ejecutado con éxito")
except Exception as e:
    print("Error en el query: {e}")

Query ejecutado con éxito


## Agregar registros a la Tabla

In [ ]:
insert_data_query = """
                INSERT INTO movies (
                    id,
                    title,
                    release_year,
                    country,
                    duration,
                    language,
                    certification,
                    gross,
                    budget
                )
                VALUES
                    (1, 'Teenage Mutant Ninja Turtles: Out of the Shadows', 2016, 'USA', 112, 'English', 'PG-13', 81638674, 135000000),
                    (2, 'The 5th Wave', 2016, 'USA', 112, 'English', 'PG-13', 34912982, 38000000),
                    (3, 'The Angry Birds Movie', 2016, 'USA', 97, 'English', 'PG', 107225164, 73000000),
                    (4, 'The BFG', 2016, 'UK', 117, 'English', 'PG', 52792307, 140000000),
                    (5, 'The Boss', 2016, 'USA', 99, 'English', 'R', 63034755, 29000000),
                    (6, 'The Boy', 2016, 'USA', 97, 'English', 'PG-13', 35794166, 10000000),
                    (7, 'The Conjuring 2', 2016, 'USA', 134, 'English', 'R', 102310175, 40000000),
                    (8, 'The Forest', 2016, 'USA', 93, 'English', 'PG-13', 26583369, 10000000),
                    (9, 'The Huntsman Winter''s War', 2016, 'USA', 120, 'English', 'PG-13', 47952020, 115000000),
                    (10, 'The Infiltrator', 2016, 'UK', 127, 'English', 'R', 14946229, 25000000),
                    (11, 'The Jungle Book', 2016, 'UK', 106, 'English', 'PG', 362645141, 175000000),
                    (12, 'The Legend of Tarzan', 2016, 'USA', 110, 'English', 'PG-13', 124051759, 180000000),
                    (13, 'The Masked Saint', 2016, 'Canada', 105, 'English', 'PG-13', 123777, 3500000),
                    (14, 'The Neon Demon', 2016, 'France', 118, 'English', 'R', 1330827, 7000000),
                    (15, 'The Perfect Match', 2016, 'USA', 96, 'English', 'R', 9658370, 5000000);
"""

try:
    cursor.execute(insert_data_query)
    print("Query ejecutado con éxito")
except Exception as e:
    print("Error en el query: {e}")

Query ejecutado con éxito


## Visualización de la Tabla Creada

In [25]:
import warnings
from tabulate import tabulate

# Ignorar advertencias específicas de pandas
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")

vista_query = "SELECT * FROM movies"

try:
    result = pd.read_sql(vista_query, conn)
except Exception as e:
    print(f"Error en el Query: {e}")

# result
# print(result.to_string(index=False))
print(tabulate(result, headers='keys', tablefmt='psql', showindex=False))

+------+-------------------------------------------------+----------------+-----------+------------+------------+-----------------+------------------+----------+
|   id | title                                           |   release_year | country   |   duration | language   | certification   |            gross |   budget |
|------+-------------------------------------------------+----------------+-----------+------------+------------+-----------------+------------------+----------|
|    1 | Teenage Mutant Ninja Turtles Out of the Shadows |           2016 | USA       |        112 | English    | PG-13           |      8.16387e+07 | 1.35e+08 |
|    2 | The 5th Wave                                    |           2016 | USA       |        112 | English    | PG-13           |      3.4913e+07  | 3.8e+07  |
|    3 | The Angry Birds Movie                           |           2016 | USA       |         97 | English    | PG              |      1.07225e+08 | 7.3e+07  |
|    4 | The BFG            

# Creación del FullText-Search

## Creación del Catálogo

In [26]:
create_catalog_query = "CREATE FULLTEXT CATALOG catalogo_movies AS DEFAULT"

try:
    cursor.execute(create_catalog_query)
    print("Catálogo creado con éxito")
except Exception as e:
    print(f"Error en el Query: {e}")

Catálogo creado con éxito


## Crear el Índice del FTS

In [27]:
create_index_query = """
                    CREATE FULLTEXT INDEX ON movies (title)
                    KEY INDEX PK__movies__3213E83F43C0E526
                    WITH STOPLIST = SYSTEM;
                    """

try:
    cursor.execute(create_index_query)
    print("Índice FTS creado con éxito")
except Exception as e:
    print(f"Error en el índice: {e}")

Índice FTS creado con éxito


# Consultas de FTS

## CONTAINS

In [32]:
contains_query="""
                SELECT * FROM movies
                WHERE CONTAINS(title, 'Demon')
            """

try:
    contains_result = cursor.execute(contains_query).fetchall()
    for row in contains_result:
        print(row)
except Exception as e:
    print(f"Error en el CONTAINS: {e}")

(14, 'The Neon Demon', 2016, 'France', 118, 'English', 'R', Decimal('1330827.00'), Decimal('7000000.00'))


## FREETEXT

In [35]:
freetext_query="""
                SELECT * FROM movies
                WHERE FREETEXT(title, 'The Neon')
            """

try:
    freetext_result = cursor.execute(freetext_query).fetchall()
    for row in freetext_result:
        print(row)
except Exception as e:
    print(f"Error en el FREETEXT: {e}")

(14, 'The Neon Demon', 2016, 'France', 118, 'English', 'R', Decimal('1330827.00'), Decimal('7000000.00'))
